# Bat gut microbio metagenomic analysis

TJ Rogers

In [2]:
import pandas as pd

## 8-Oct-2024

* To do:
  - [X] Find good host sequences so that I can remove host reads from the metagenomes.
  - [X] Run snakemake workflow to trim reads and retrieve host reference sequences.
  - [X] Create script to remove host reads from the fastq files.
  - [X] Update the main `README.md` file
  - [ ] Check the reference file to see if they all end in "_mPhyDis1.pri.v3_genomic.fna". If not, need to figure out how to reference them in the params section of the rule remove_host_reads in `readQC.smk`*

### Find good host sequences so that I can remove host reads from the metagenomes.

Below, I have listed the sample name (host) along with the species name and the accession number for the reference seuquence:

In [3]:
metadata={'sample': ['NBS1051F-30-542494038','NBS1051F','NBS1079E-30-542494038','NBS1079E-30-550131958','080613-8-pd','080813-19-pd','080813-7-mc','080913-2-mc','080913-7-ph'],
          'species': ['Pteronotus parnellii','Pteronotus parnellii','Myotis elegans', 'Myotis elegans','Phyllostomus discolor','Phyllostomus discolor','Mimon crenulatum','Mimon crenulatum','Phyllostomus hastatus'],
          'accession': ['GCA_036768555.1','GCA_036768555.1','NaN','NaN','GCA_004126475.3','GCA_004126475.3','NaN','NaN','GCA_019186645.2']}

# Create DataFrame
metadata = pd.DataFrame(metadata)
print(metadata)

                  sample                species        accession
0  NBS1051F-30-542494038   Pteronotus parnellii  GCA_036768555.1
1               NBS1051F   Pteronotus parnellii  GCA_036768555.1
2  NBS1079E-30-542494038         Myotis elegans              NaN
3  NBS1079E-30-550131958         Myotis elegans              NaN
4            080613-8-pd  Phyllostomus discolor  GCA_004126475.3
5           080813-19-pd  Phyllostomus discolor  GCA_004126475.3
6            080813-7-mc       Mimon crenulatum              NaN
7            080913-2-mc       Mimon crenulatum              NaN
8            080913-7-ph  Phyllostomus hastatus  GCA_019186645.2


* I already have the script written to download sequences NCBI, but the genome of some host have not been sequenced and are not found on any data base. For instance, neither Myotis elegans nor Mimon crenulatum have had their genomes sequences.
  * However, according to Hurtado and D'Elia-2018, Mimon crenulatum is not an actual species. It is actually a clade that is composed of multiple species. One such species is M. crenulatum keenani. I found that the database 'bat1k' has the genome of M. crenulatum keenani sequenced. So I will retrieve the sequence from there. 
  * I am currently waiting on bat1k to approve my membership so I can download the sequence data I need.

### Run snakemake workflow to trim reads and retrieve host reference sequences.


* JobID: 11161798:
  * Apparently this version of snakemake can not take in bash script in the arguement "script:". I got the following error:
    * "Unsupported script: Expecting either Python (.py), R (.R), RMarkdown (.Rmd) or Julia (.jl) script."

  * I added the script path to "params" and changed the argument "script" to "shell". Going to try again.

* JobID: 11161965
  * I made a mistake by giving the same output read file for the fr1 and fr2 slot in the in the shell section of the clean_raw_reads rule in the file readQC.smk. 
  * Made the required change and running again.

* JobID: 11162659
  * For most of the samples, the adapter and phix removal worked. However, for some reason the temp fq file for sample 080913-2-mc isnt being created correctly or some other error is happening with it as snakemake is removing it and faulting out.
  * I am troubleshooting now:
    * Going to set up the rule to where it will create a log so I can use that to hopefully troubleshoot this sample.

* JobID: 11163418
  * I noticed that I was having the script place the trimmed reads in the 'tmp' directory. I have changed that in the script. 
  * Once this job has stopped I need to:
    - [x] mv the trimmed fq files to the clean_reads directory.
  * This job failed as well for the same reason as the previous job. It said that line 23 in `trimming.sh` was not a command. Problem is I had already modified that script and was unable to figure out what line it was talking about. I am trying to run again to see what will happen.

* JobID: 11163492
  * Job completed without any issues as of 09-Oct-2024

### Create script to remove host reads from the fastq files.

* While I am currently creating this script, it will not be able to be tested until bat1k has given me access to their data base.
  * Script name: `host_read_removal.sh`
  * path: `workflow/scripts/`

## 9-Oct-2024

* To do:
  - [X] Check email to see if Bat1k has granted you access to download their sequences.
  - [X] Find missing host reference sequences or good alternatives. 
  - [X] Check status of JobID 11163492 and update on previous day's notes.
  - [X] Read lit papers in reference to bats, bat microbiomes, and HiC ananysis.
  - [ ] ~~Write script for assembling reads.~~
  - [X] Make needed adjustments to the `host_read_removal.sh`.
  - [X] Update the main `README.md` file
  - [X] Check the reference file to see if they all end in "_mPhyDis1.pri.v3_genomic.fna". If not, need to figure out how to reference them in the params section of the rule remove_host_reads in `readQC.smk`

### Check email to see if Bat1k has granted you access to download their sequences.

* Bat1k has not sent me an update as of yet. I will wait until this afternoon. If they have not gotten a hold of me by that time, I will reach out to them again.
  * I found replacement reference sequences on NCBI and no longer need access to Bat1k.

### Find missing host reference sequences or good alternatives. 


* Myotis elegans does not have a whole genome sequences. However, Stadelmann et al. 2007 and Moratell et al. 2013 suggest that M. elegans closest relatives are (in order of closest to least) M. simus, M. riparius, M. ruber, and M. keaysi. This gives me some options to look for. I will check the NCBI data base to see if there are seuqences for any of these to use them as a reference genome. If not, I will work my way further down the tree until i get a hit.
  * None of the above listed options have had their genome sequenced. Howerever, I did find two genomes from members within the same clade: M. yumanesis and M. vivesi. M. vivesi is larger and eats fish as well as insects, whereas M. yumanesis seems to be more like M. elegans in size and diet. So I am going to use M. yumanesis as a stand in.
* Mimon crenulatum (Gardnerycteris crenulatum) does not have a whole genome sequenced. However, it turns out that Phyllostomus discolor is a close relative of M. crenulatum (Hurtado et. al, 2018). So I will use it as a reference genome.

* Below, I have updated the metatable to reflect new additions to the reference sequences:

In [5]:
metadata2={'sample': ['NBS1051F-30-542494038','NBS1051F','NBS1079E-30-542494038','NBS1079E-30-550131958','080613-8-pd','080813-19-pd','080813-7-mc','080913-2-mc','080913-7-ph'],
          'species': ['Pteronotus parnellii','Pteronotus parnellii','Myotis elegans', 'Myotis elegans','Phyllostomus discolor','Phyllostomus discolor','Mimon crenulatum','Mimon crenulatum','Phyllostomus hastatus'],
          'accession': ['GCA_036768555.1','GCA_036768555.1','GCA_028538775.1','GCA_028538775.1','GCA_004126475.3','GCA_004126475.3','GCA_004126475.3','GCA_004126475.3','GCA_019186645.2'],
          'substitute_species': ['none','none','Myotis yumanensis','Myotis yumanensis','none','none','Phyllostomus discolor','Phyllostomus discolor','none']}

# Create DataFrame
metadata2 = pd.DataFrame(metadata2)
metadata2

,sample,species,accession,substitute_species
0,NBS1051F-30-542494038,Pteronotus parnellii,GCA_036768555.1,none
1,NBS1051F,Pteronotus parnellii,GCA_036768555.1,none
2,NBS1079E-30-542494038,Myotis elegans,GCA_028538775.1,Myotis yumanensis
3,NBS1079E-30-550131958,Myotis elegans,GCA_028538775.1,Myotis yumanensis
4,080613-8-pd,Phyllostomus discolor,GCA_004126475.3,none
5,080813-19-pd,Phyllostomus discolor,GCA_004126475.3,none
6,080813-7-mc,Mimon crenulatum,GCA_004126475.3,Phyllostomus discolor
7,080913-2-mc,Mimon crenulatum,GCA_004126475.3,Phyllostomus discolor
8,080913-7-ph,Phyllostomus hastatus,GCA_019186645.2,none


### Check status of JobID 11163492 and update on previous day's notes.

* Job is complete. References have been downloaded. Only issue is that I did not inlcude a command in the script to unzip the .zip file. Updating and running that portion again.

* JobID:11188202
  * This job failed. Not sure what the issue was. But I have split the rule 'get_host_references' into two: 'get_host_references' and 'decompress_references'

* JobID: 11188332
  * Job complete. Now to test if the decompress_references rule works. First, I will need to unzip the ncbi file to see what the common file path is.

* JobID: 11189456 (Testing unzip on ncbi reference file)
  * In this, I also added a find command to find all the '.fna' files to clean up their names
    * IT WORKED!!!!

* JobID: 11189989
  * I am now rerunning the get_host_references and decompress_references rules in the snakemake pipeline. 

## 10-Oct-2024

There has been a change of plans. I spoke to Rick today about the above pipeline. He pointed out that the lab's newest program, Titian, already does everything up to assembling the reads. It can even take in host files to use as read contamination control. The only thing it doesnt do yet is microbial binning. Rick has asked me to constuct the script to do the binning part. This gives me the oprotunity to learn more python. So, I need to do the following:
* [X] Check over the Titian output that Josh Hensley has already produced. It is found under `/projects/raw_lab/projects/yohe_lab_bats/`.
* [X] Find out if the contamination control part of Titain is up and running.
* [X] Find out if Titian is making a co-assembly or if it makes individual assemblies:
  * [X] If it makes individual assemblies, good. We can press on.
  * [ ] ~~If it makes one co-assembly, talk to Jose about making it optional.~~
* [ ] Write out python script to be used in Titian that bins the assembies.
  * [ ] Find out which binners would be best to use.
  * [ ] Build python script to bin the assemblies. Steps to include are as follows:
    * [ ] Assemble each sample individually
    * [ ] Bin each assembly individually using 3 different binning programs:
      * [ ] In this step, is if it would be benifical to map all sample reads to each assembly to add in binning (Some binning programs already have this ability).
    * [ ] Use metawrap's bin refinner to create an individual refined bin set for each sample 
    * [ ] Use Drep to depreplicate the refined bin sets so that we are left with one single bin set


## 11-Oct-2024

* To do:
  - [X] Assemble each sample individually
    * Titan already does seperate assemblies. This is good!
  - [X] Write python script to bin each assembly:
    - [ ] Bin each assembly individually using 3 different binning programs:
        - [ ] In this step, is if it would be benifical to map all sample reads to each assembly to add in binning (Some binning programs already have this ability).
    - [ ] Use metawrap's bin refinner to create an individual refined bin set for each sample 
    - [ ] Use Drep to depreplicate the refined bin sets so that we are left with one single bin set


* Running titan
  * JobID:11218747
    * When this job finishes, I need to try to run my binning script on it.
    * This job completed without issue. However, I did not deconcaminate the host DNA. This was also ran using megahit. I am currently running it again. There are 4 host sequences but Titan only takes in one fasta file at a time. So I am going to concatenate the four together and use them as one single input. Check next day for job running ID

## 14-Oct-2024

To do:
- [ ] complete script to map create read depth file for assemblers
- [ ] complete binning functions
- [ ] finalize peer review

* Running Titan again
  * JobID: 11226526
    * Running Titan using both megahit and metaspades while also decontaminating
    * For some reason this job failed. I believe it may be due to the contamination file I was using. So I am going to run again without the contamination.
  * JobID:11226750
* Testing mapping function for my binning script.
  * JobID: 11242127
    * I had to do a lot of troubleshooting, but its finally working (it looks like at least)
    * Job failed do to not enough memory. See next day run for update

## 15-Oct-2024

To Do:
*   Read over paper you need to review
*   Work on binning script
    *   Check to see Jose settings in his script for bwa
*   Check Phasegenome

* Testing mapping function for my binning script.
  * JobID: 11271686
    * The last job failed due to not having enough memmory so I changed the following parameter in my sbatch script:
      * #SBATCH --mem=200gb to #SBATCH --mem=400gb
        * if this does not work, I will follow the advice given in this link:
          * https://stackoverflow.com/questions/52421068/error-in-slurm-cluster-detected-1-oom-kill-events-how-to-improve-running-jo
    * Job failed. Rick just wants me to create bins and move away from the scripting. So I am moving away from this for now and am running a job to make the mags
  * JobID: 11288072

## 16-Oct-2024

- The following are in the email titled Important Biosafety training to be completed and occupational health registration
Inbox
  - [ ] Complete the following CBTs:
    - [ ] Laboratory Personnel BSS
    - [ ] Shipping and Transport of Regulated Biological Materials
    - [ ] Animal Biosafety
  - [ ] Complete review of the Information on Rabies Virus and Vaccination Google form
  - [ ] Once all this required training is completed, complete the Biohazard Awareness Training 
- [X] Complete binning script
- [ ] Complete the Active Assailant Response Training
- [ ] Finish reviewing paper.

There were a number of problems with the binning script:
*   First, the error was in regards to anvio not liking "-" in the prefix names. So, I changed all the assembly names to have underscores. This allowed for the prefix to have underscorse as well.
*   Next, anvio through an error because one of the assemblies started with a number. So, I added the prefix "sample_" to it. This solved that problem.
*   Next, metawrap will not take gunziped files even though the programs it is using can use gunziped files. So, I modified the binning.sh script of metawrap. I changed all occurances of "_1.fastq" to "_R1.fastq.gz". This solved that problem.
*   Finally, an older version of samtools has priority over other versions on HPC. This caused a conflict with metawrap as it uses a newer version "1.9". So, after trouble shooting, I made sure to load the module for samtools 1.9 in the binning script.

JobID 11327447:
*   running as of 1550


## 18-Oct-2024

*   was only able to make a few bins from the Bat Gut data using metawrap. I am going to move on to the Green lake data and see if I can get the pipeline to work. Then I will come back here if it works.